In [ ]:
Cell 1 – Setup & Imports
# Install packages if needed (Colab)
# !pip install pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: configure pandas display
pd.set_option('display.max_columns', None)


In [ ]:
Cell 2 – Load Data
# Replace with your dataset path or Colab upload
data_path = 'path/to/your/data.csv'
data = pd.read_csv(data_path)

# Quick check of the first few rows
data.head()


In [ ]:
Cell 3 – Data Exploration
# Number of records
num_records = data.shape[0]
print("Number of records:", num_records)

# Number of individuals with income >50k
num_income_gt_50k = (data['income'] == '>50K').sum()
print("Number of individuals with income >50K:", num_income_gt_50k)

# Number of individuals with income <=50k
num_income_le_50k = (data['income'] == '<=50K').sum()
print("Number of individuals with income <=50K:", num_income_le_50k)

# Percentage with income >50k
pct_income_gt_50k = (num_income_gt_50k / num_records) * 100
print("Percentage of individuals with income >50K: {:.2f}%".format(pct_income_gt_50k))


In [ ]:
Cell 4 – Data Preprocessing
# Separate features and target
X = data.drop('income', axis=1)
y = data['income']

# One-hot encode categorical variables
categorical_features = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, drop='first')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_features]), columns=encoder.get_feature_names_out(categorical_features))

# Combine with numeric features
X_numeric = X.select_dtypes(include=[np.number])
X_preprocessed = pd.concat([X_numeric.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1)

# Encode target
y_encoded = y.map({'>50K': 1, '<=50K': 0})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.2, random_state=42)


In [ ]:
Cell 5 – Naive Predictor Performance
# Predict all >50K (naive)
y_naive_pred = np.ones_like(y_test)

accuracy_naive = accuracy_score(y_test, y_naive_pred)
f1_naive = f1_score(y_test, y_naive_pred)

print("Naive Predictor Accuracy: {:.4f}".format(accuracy_naive))
print("Naive Predictor F1 Score: {:.4f}".format(f1_naive))


In [ ]:
Cell 6 – Model Selection & Pros/Cons
# List models to explore
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'AdaBoost': AdaBoostClassifier()
}

# Here, add a markdown cell or comment with pros/cons of each model
# Example:
# Logistic Regression: fast, interpretable, good baseline, may underfit
# Random Forest: handles nonlinear relationships, feature importance available, more computational cost
# AdaBoost: robust to overfitting, good with weak learners, slower with large data
# References: [Add your sources here]


In [ ]:
Cell 7 – Training & Evaluation Pipeline
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results[name] = {'Accuracy': acc, 'F1 Score': f1}

# Display results
results_df = pd.DataFrame(results).T
results_df


In [ ]:
Cell 8 – Performance Visualization
results_df.plot(kind='bar', figsize=(10,6))
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.show()


In [ ]:
Cell 9 – Model Tuning (Grid Search)
# Example: tune Random Forest max_depth
param_grid = {'max_depth': [5, 10, 15]}

grid = GridSearchCV(RandomForestClassifier(), param_grid, scoring='f1', cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)

print("Best parameters:", grid.best_params_)
print("Optimized Model Accuracy:", accuracy_score(y_test, y_pred_best))
print("Optimized Model F1 Score:", f1_score(y_test, y_pred_best))


In [ ]:
Cell 10 – Feature Importance
# Feature importances for Random Forest
importances = best_model.feature_importances_
feature_names = X_preprocessed.columns
feat_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feat_imp_df = feat_imp_df.sort_values(by='Importance', ascending=False)

# Top 5 features
top5_features = feat_imp_df.head(5)
print(top5_features)

# Optional plot
sns.barplot(x='Importance', y='Feature', data=top5_features)
plt.title('Top 5 Feature Importances')
plt.show()


In [ ]:
Cell 11 – Effects of Feature Selection
# Train model using only top 5 features
X_train_top5 = X_train[top5_features['Feature']]
X_test_top5 = X_test[top5_features['Feature']]

best_model.fit(X_train_top5, y_train)
y_pred_top5 = best_model.predict(X_test_top5)

print("Top 5 Features Model Accuracy:", accuracy_score(y_test, y_pred_top5))
print("Top 5 Features Model F1 Score:", f1_score(y_test, y_pred_top5))
